<a id = "1"></a><p style = "font-size : 40px; color :#E7E8D1 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #B85042; border-radius: 5px 5px;"><strong>SECTION 2 - DOWNLOAD DATASET</strong></p> 

<a id = "1"></a><p style = "font-size : 40px; color :#E7E8D1 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #B85042; border-radius: 5px 5px;"><strong>1. LIBRARIES</strong></p>

In [1]:
from time import sleep
from selenium.webdriver.common.by import By
from seleniumbase import Driver
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import math
from tqdm import tqdm
import shutil
import sys
import warnings
from PIL import Image
if not sys.warnoptions:
    warnings.simplefilter("ignore")

<a id = "1"></a><p style = "font-size : 40px; color :#E7E8D1 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #B85042; border-radius: 5px 5px;"><strong>2. DOWNLOADING IMAGES USING MULTITHREADING</strong></p>

# Setup

In [2]:
# Get Df from section-1 to download images
from config import final_file_path, df_name, project_root
df = pd.read_excel(final_file_path)
print(f'Df name  : {df_name}')
print(f'Df length: {len(df)}')

# Create a Download Folder for Images
downloaded_images = f"{project_root}/downloaded_images/" # commonly used folder path in the project
os.makedirs(downloaded_images, exist_ok=True)

# Lists and Configuration
ok_images = list()
not_ok_images = list()
white_images = list()
list_id_not_ok = []
df["path"] = ""
not_downloaded_for_selenium = list()
img_url_base = "https://cdn.eksisozluk.com/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "https://eksisozluk.com/",
}

Df name  : 2_dataset
Df length: 37


# Download

In [3]:
def download_image(i):
    img_url = df['img_url'][i]
    img_index = df['img_index'][i]
    entry_id = df['entry_id'][i]
    
    response = requests.get(img_url, headers=headers)
    
    if response.status_code == 200:  # 1- If 200: Download
        img_path = os.path.join(downloaded_images, f"img_{img_index}.jpg")
        with open(img_path, "wb") as file:
            file.write(response.content)
            
    elif response.status_code == 404:  # 2- If 404: Create edit date URL and try downloading again
        status_code = response.status_code
        img_url_date = df['entry_edit_date'][i]
        url_id = img_url.split('/')[-1].split('.')[0]
        
        if pd.notna(img_url_date):
            img_url_date = str(img_url_date)
            date_object = datetime.strptime(img_url_date, "%d.%m.%Y %H:%M")
            entry_day = str(date_object.day)
            entry_month = str(date_object.month)
            entry_year = str(date_object.year)
            img_url = img_url_base + entry_year + '/' + entry_month + '/' + entry_day + '/' + url_id[0] + '/' + url_id + '.jpg'
            response = requests.get(img_url, headers=headers)
            
            if response.status_code != 200:  # 3- If not 200: Try with PNG
                img_url = img_url_base + entry_year + '/' + entry_month + '/' + entry_day + '/' + url_id[0] + '/' + url_id + '.png'
                response = requests.get(img_url, headers=headers)
                
                if response.status_code != 200:  # 4- If not 200: Try edit date URL with one day earlier
                    date_object -= timedelta(days=1)
                    entry_year = date_object.strftime("%Y")
                    entry_month = str(int(date_object.strftime("%m")))  # Not starting with 0
                    entry_day = str(int(date_object.strftime("%d")))    # Not starting with 0
                    img_url = f"{img_url_base}{entry_year}/{entry_month}/{entry_day}/{url_id[0]}/{url_id}.jpg"
                    response = requests.get(img_url, headers=headers)
                    if response.status_code != 200: # 5- If not 200: Try one-day-earlier edit date URL with PNG
                        img_url = img_url_base + entry_year + '/' + entry_month + '/' + entry_day + '/' + url_id[0] + '/' + url_id + '.png'
                        response = requests.get(img_url, headers=headers)
                        
        else:
            img_url = img_url.replace('jpg','png')
            response = requests.get(img_url, headers=headers)
            
        if response.status_code == 200:  # 6- If 200: Download
            img_path = os.path.join(downloaded_images, f"img_{img_index}.jpg")
            #ok_images.append(img_index)
            with open(img_path, "wb") as file:
                file.write(response.content)
                
    else:
        print(f'response.status_code is {response.status_code} for img_index={img_index}')
        
    # Add status_code to xlsx file
    df.at[img_index-1, "response_status_code"] = response.status_code
    
    # Add img_index by looking statuscode
    if response.status_code != 200:
        not_downloaded_for_selenium.append(img_index)
    else:
        ok_images.append(img_index)
        
    # Write img_path to df
    img_path = os.path.join(downloaded_images, f"img_{img_index}.jpg")
    filename = img_path.split("\\")[-1]
    df.loc[i, "path"] = filename
    if response.status_code == 200:
        filename = img_path.split("\\")[-1]
        
# Multithreading | Set up a ThreadPoolExecutor to download images concurrently
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = [executor.submit(download_image, i) for i in range(0, len(df))]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading images"):
        future.result()  # Waiting for all tasks to complete

# Save final df & 404 df
df.to_excel(f"{df_name}_without_selenium_status_{len(df)}.xlsx", index=False)
dataset_without_selenium_404 = df[df["response_status_code"] != 200]
dataset_without_selenium_404.to_excel(f"3_dataset_status_404.xlsx", index=False)

# Result

In [4]:
# Final result
print("\n---All operations finished. Final file saved---")
print(f"Final file name: {df_name}_without_selenium_status_{len(df)}.xlsx")
print(f'Final Result:')
print(f'1) df lenght: {len(df)}')
print(f'2) ok_images lenght: {len(ok_images)}')
print(f'3) not_ok_images lenght: {len(not_ok_images)}')
not_downloaded_for_selenium.sort()
print(f'4) not_downloaded_for_selenium lenght: {len(not_downloaded_for_selenium)}')
print(f'5) not_downloaded_for_selenium indexes: {[int(index) for index in not_downloaded_for_selenium]}')


---All operations finished. Final file saved---
Final file name: 2_dataset_without_selenium_status_37.xlsx
Final Result:
1) df lenght: 37
2) ok_images lenght: 37
3) not_ok_images lenght: 0
4) not_downloaded_for_selenium lenght: 0
5) not_downloaded_for_selenium indexes: []


<a id = "1"></a><p style = "font-size : 40px; color :#E7E8D1 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #B85042; border-radius: 5px 5px;"><strong>2. DOWNLOADING IMAGES USING SELENIUM</strong></p>

# Create Img List

In [5]:
# Identify the images to check with Selenium

# 1- Read all .jpg files in the folder
all_files = os.listdir(downloaded_images)
jpg_files = [f for f in all_files if f.endswith('.jpg')]

# 2- Extract existing file numbers
existing_numbers = set()
for filename in jpg_files:
    try:
        number_part = int(filename.replace("img_", "").replace(".jpg", ""))
        existing_numbers.add(number_part)
    except ValueError:
        pass  # ignore files with incorrect names

# 3- All expected numbers from 1 to the last index
full_set = set(range(1, len(df)+1))  # 69431 dahil olsun diye 69432 yazdım

# 4- Missing files
missing_numbers = full_set - existing_numbers
print(f"Total number of missing images: {len(missing_numbers)}")

# 5- Read the Excel file
df = pd.read_excel(final_file_path)

# 6- Keep only the img_index values of the missing files
df_filtered = df[df['img_index'].isin(missing_numbers)]

# 7- Save as a new file
from config import project_root
path_output = os.path.join(project_root, '4_dataset_missing_images.xlsx')
df_filtered.to_excel(path_output, index=False)

print(f"Missing img_index values saved to '{path_output}'")

Total number of missing images: 0
Missing img_index values saved to 'C:\Users\soner.kar\Desktop\My Projects\SözlükLens\4_dataset_missing_images.xlsx'


# Run Selenium

In [6]:
downloaded_images_selenium = f"{project_root}/downloaded_images_selenium/"
os.makedirs(downloaded_images_selenium, exist_ok=True)

df_missings = pd.read_excel(path_output)
not_downloaded_for_selenium  = df_missings['img_index'].tolist()
####### df = pd.read_excel(r"C:\Users\soner.kar\Desktop\My Projects\2- Ekşi Sözlük Sözlük Yazarlarının Çektiği Fotoğraflar\dataset_69431_updated.xlsx")
df = df_missings

not_ok_images_selenium = list()
ok_images_selenium = list()

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "https://eksisozluk.com/",
}

print(f"Number of images to retry with Selenium: {len(not_downloaded_for_selenium)}")
#print('Images index to retry with Selenium: ', not_downloaded_for_selenium)
# Go to Ekşi Sözlük login page
img_url_base = "https://cdn.eksisozluk.com/"
# Start Selenium WebDriver
print('Selenium will run now...')
driver = Driver(uc=True)
driver.maximize_window()
url_login = "https://eksisozluk.com/giris"
driver.uc_open_with_reconnect(url_login, 10)
sleep(1)
counter = 0
for i in range(len(not_downloaded_for_selenium)):
    counter += 1
    if counter %100 == 0:
        print('checkpoint at counter: ', counter); print('len(ok_images_selenium) : ', len(ok_images_selenium))
    img_url = df['img_url'][i]
    img_index = df['img_index'][i]
    entry_id = df['entry_id'][i]
    url_id = img_url.split('/')[-1].split('.')[0]
    try:
        driver.get("https://soz.lk/i/" + url_id)
        sleep(2)
        image_element = driver.find_element(By.ID, "image")
        image_src = image_element.get_attribute("src")
        response = requests.get(image_src, headers=headers)
        if response.status_code == 200:
            img_path = os.path.join(downloaded_images_selenium, f"img_{img_index}.jpg")
            with open(img_path, "wb") as file:
                file.write(response.content)
            filename = img_path.split("\\")[-1]
            ok_images_selenium.append(img_index)
            df.at[i+1, "path"] = filename
            df.at[i+1, "response_status_code"] = response.status_code
        else:
            not_ok_images_selenium.append(img_index)
            df.at[i+1, "response_status_code"] = response.status_code
    except Exception as e:
        not_ok_images_selenium.append(img_index)

print("Selenium has finished running.")
print('\n-Result-')
print('Lenght of Ok Images: ', len(ok_images_selenium))
print('Lenght of Not Ok Images: ', len(not_ok_images_selenium))

Number of images to retry with Selenium: 0
Selenium will run now...
Selenium has finished running.

-Result-
Lenght of Ok Images:  0
Lenght of Not Ok Images:  0


# Clear XLSX 

In [7]:
# Match images in the folder with those in the xlsx file and remove unmatched rows from the xlsx

# 1- Read all .jpg files in the folder
all_files = os.listdir(downloaded_images)
jpg_files = [f for f in all_files if f.endswith('.jpg')]

# 2- Extract existing file numbers
existing_numbers = set()
for filename in jpg_files:
    try:
        number_part = int(filename.replace("img_", "").replace(".jpg", ""))
        existing_numbers.add(number_part)
    except ValueError:
        pass  # ignore if there's an error in the filename

# 3- Read the Excel file
df = pd.read_excel(final_file_path)

# 4- Keep only rows with existing img_index values
df_filtered = df[df['img_index'].isin(existing_numbers)]

# 5- Save the new file
from config import img_index ###---------------------------------------------------------------------------------------------###
path_output = os.path.join(project_root, f"5_dataset_only_existing_{img_index}.xlsx")
df_filtered.to_excel(path_output, index=False)

print(f"Done. Kept only rows with images.\nNew file: '{path_output}'")

Done. Kept only rows with images.
New file: 'C:\Users\soner.kar\Desktop\My Projects\SözlükLens\5_dataset_only_existing_37.xlsx'


# Rename Images and Update XLSX

In [8]:
# Rename images and update the img_index column in the xlsx file accordingly

# 1- Paths
final_file_path = path_output

# 2- Read all .jpg files
jpg_files = [f for f in os.listdir(downloaded_images) if f.endswith('.jpg')]

# 3- Extract numbers from file names
img_numbers = []
file_map = {}  # old filename -> number

for file in jpg_files:
    try:
        number = int(file.replace('img_', '').replace('.jpg', ''))
        img_numbers.append(number)
        file_map[number] = file
    except:
        pass

# 4- Sort image numbers
img_numbers_sorted = sorted(img_numbers)

# 5- Read the Excel file
df = pd.read_excel(final_file_path)

# 6- Select rows with img_index values matching images in the folder
df_filtered = df[df['img_index'].isin(img_numbers_sorted)].copy()

# 7- Create a new dataframe:
# img_1.jpg → take the row with the old number, set img_index = 1
new_rows = []

for new_idx, old_number in enumerate(img_numbers_sorted, start=1):
    row = df_filtered[df_filtered['img_index'] == old_number]
    if not row.empty:
        updated_row = row.copy()
        updated_row.loc[:, 'img_index'] = new_idx  # only update img_index
        new_rows.append(updated_row)

# 8- Combine all new rows
df_final = pd.concat(new_rows, ignore_index=True)

# 9- Rename old files according to the new order
for new_index, old_number in enumerate(img_numbers_sorted, start=1):
    old_filename = file_map[old_number]
    old_path = os.path.join(downloaded_images, old_filename)
    new_filename = f"img_{new_index}.jpg"
    new_path = os.path.join(downloaded_images, new_filename)
    
    if old_path != new_path:
        os.rename(old_path, new_path)

# 10- Save the updated Excel file
output_final_file_path = os.path.join(project_root, "6_dataset_final.xlsx")
df_final.to_excel(output_final_file_path, index=False)

print(f"Images and Excel data ('{os.path.basename(output_final_file_path)}') updated with new order.")

Images and Excel data ('6_dataset_final.xlsx') updated with new order.
